In [1]:
import numpy as np
from mha021 import *

In [2]:
# Problem 1
# a)
print('a)')

x1 = 0.014
y1 = 0.010
x2 = 0.021
y2 = 0.009
x3 = 0.018
y3 = 0.018
x4 = 0.012
y4 = 0.016

xi_1 = 0
xi_2 = 1

N1 = (1 - xi_1) * (1 - xi_2) / 4
N2 = (1 + xi_1) * (1 - xi_2) / 4
N3 = (1 + xi_1) * (1 + xi_2) / 4
N4 = (1 - xi_1) * (1 + xi_2) / 4

N_bar = np.array([
    N1, N2, N3, N4
])

x_e = np.array([
    [x1],
    [x2],
    [x3],
    [x4]
])

y_e = np.array([
    [y1],
    [y2],
    [y3],
    [y4]
])

x_A = N_bar @ x_e
y_A = N_bar @ y_e

print(x_A)
print(y_A)

# b)
print('b)')

scale = 1e-5
u1 = np.array([4.1, 1.0]) * scale
u2 = np.array([3.5, 1.6]) * scale
u3 = np.array([3.8, 1.7]) * scale
u4 = np.array([3.7, 1.5]) * scale

a_e = np.array([
    u1[0],
    u1[1],
    u2[0],
    u2[1],
    u3[0],
    u3[1],
    u4[0],
    u4[1]
])

N_e = np.array([
    [N1, 0, N2, 0, N3, 0, N4, 0],
    [0, N1, 0, N2, 0, N3, 0, N4]
])

u_A = N_e @ a_e
print(u_A)

a)
[0.015]
[0.017]
b)
[3.75e-05 1.60e-05]


In [3]:
# Problem 2

#a)
print('a)')

# Define inputs
E = 1
A = 1
I = 1
rho = 1
P = 1
L = 1

Ex = np.array([
    [0, 2 * L],
    [2 * L, 0]
])

Ey = np.array([
    [0, L],
    [L, 2 * L]
])

fig = draw_discrete_elements(
    Ex, Ey
)
fig.show()

"Edof can't be used since there is a bar and a beam"
# Edof = np.array([
#     [1,2,3,4,5,6],
#     [4,5,6,7,8,9]
# ])

nel = 2
ndofs = 8

# Initiate stiffness matrix and load vector
K = np.zeros((ndofs, ndofs))
f = np.zeros((ndofs))

# Element 1: beam
ex1 = Ex[0, :]
ey1 = Ey[0, :]
Ke1 = beam2e(ex1, ey1, E, A, I)
dofs1 = [1,2,3,4,5,6]
assem(K, Ke1, dofs1)

# Element 2: bar
ex2 = Ex[1, :]
ey2 = Ey[1, :]
Ke2 = bar2e(ex2, ey2, E, A)
dofs2 = [4,5,7,8]
assem(K, Ke2, dofs2)

# Force P
f[5-1] = - P

# Boundary conditions
bc_dofs = np.array([1, 2, 3, 4, 7, 8])
bc_vals = np.array([0, 0, 0, 0, 0, 0])

# Solve system
a, r = solve_eq(K, f, bc_dofs, bc_vals)

# Displacement at point C
displacement_C = a[5 - 1]
print(displacement_C)

# b)
print('b)')

# Initiate mass matrix
M = np.zeros((ndofs, ndofs))

M1 = beam2m(ex1, ey1, rho, A)
assem(M, M1, dofs1)

M2 = bar2m(ex2, ey2, rho, A)
assem(M, M2, dofs2)

# Free degrees of freedom
free_dofs = [5, 6]

# Reduced matrices
K_red = extract_block(K, free_dofs)
M_red = extract_block(M, free_dofs)

# Solve eigenvalue-problem
omega2, phi = eigh(K_red, M_red)

print(f'smallest eigenfrequency [Hz]: {np.min(np.sqrt(omega2) / (2 * np.pi)):.4f}')

a)


-2.5409863380679423
b)
smallest eigenfrequency [Hz]: 0.1169


In [4]:
# Problem 3

# a)
print('a)')

E = 200e9
A = 2e-5
b0 = 10e3
L = 1

def b(x):
    return b0 * x / L
    
b1 = b(L/6)
b2 = b(3*L/6)
b3 = b(5*L/6)

print(b1, b2, b3)

fl1 = b1
fl2 = 1/2 * (b1 + b2)

num_elements = 3

Le = L / num_elements
k_e = (E * A) / Le

num_nodes = num_elements + 1
K = np.zeros((num_nodes, num_nodes))
f = np.zeros(num_nodes)

for i in range(num_elements):
    n1 = i
    n2 = i + 1
    
    K[n1, n1] += k_e
    K[n1, n2] -= k_e
    K[n2, n1] -= k_e
    K[n2, n2] += k_e
    
    x_mid = (n1 * Le + n2 * Le) / 2
    
    b_val = b0 * (x_mid / L)
    
    # Local load vector contribution
    f_node_val = b_val * Le / 2
    
    f[n1] += f_node_val
    f[n2] += f_node_val
    
# d)
print(b0 / 2 * L)

bc_dofs = [1]
bc_vals = [0]

a, r = solve_eq(K, f, bc_dofs, bc_vals)
print(a)

print(E * A / (L / 3) * a[2-1])

a)
1666.6666666666665 5000.0 8333.333333333334
5000.0
[0.         0.00039352 0.00069444 0.00081019]
4722.222222222221


In [ ]:
# Problem 4

# a)
print('a)')

xi, eta = sp.symbols('xi, eta')

x_e = sp.Matrix([
    [0.014],
    [0.021],
    [0.018],
    [0.012]
])

y_e = sp.Matrix([
    [0.010],
    [0.009],
    [0.018],
    [0.016]
])

N1 = 1/4 * (1 - xi) * (1 - eta)
N2 = 1/4 * (1 + xi) * (1 - eta)
N3 = 1/4 * (1 + xi) * (1 + eta)
N4 = 1/4 * (1 - xi) * (1 + eta)

N_bar = sp.Matrix([
    [N1, N2, N3, N4]
])

x = N_bar @ x_e
y = N_bar @ y_e

Matrix = sp.Matrix([x, y])
Variables = sp.Matrix([xi, eta])

J = Matrix.jacobian(Variables)

J_func = sp.lambdify((xi, eta), J, 'numpy')

qr_points_1D = np.array([-0.577350269, +0.577350269])
qr_points_2D = np.zeros((len(qr_points_1D), len(qr_points_1D), 2))

for i in range(len(qr_points_1D)):
    for j in range(len(qr_points_1D)):
        qr_points_2D[i, j] = [qr_points_1D[i], qr_points_1D[j]]

A=0
for i in range(2):
    for j in range(2):
        xi, eta = qr_points_2D[i,j,:]
        J = J_func(xi, eta)
        A += np.linalg.det(J)
        
print(A)

# b)
print('b)')

def compute_Ne_Be_detJ(nodes, xi_vals, eta_vals):
    
    x_e = nodes[:, 0]
    y_e = nodes[:, 1]
        
    xi, eta = sp.symbols('xi, eta')

    N1 = 1/4 * (1 - xi) * (1 - eta)
    N2 = 1/4 * (1 + xi) * (1 - eta)
    N3 = 1/4 * (1 + xi) * (1 + eta)
    N4 = 1/4 * (1 - xi) * (1 + eta)

    N_bar = sp.Matrix([
        [N1, N2, N3, N4]
    ])

    x = N_bar @ x_e
    y = N_bar @ y_e

    J = sp.Matrix([x, y]).jacobian(sp.Matrix([xi, eta]))

    dN_bar_dxi = sp.diff(N_bar, xi)
    dN_bar_deta = sp.diff(N_bar, eta)
    
    J_func = sp.lambdify((xi, eta), J, 'numpy')
    dN_bar_dxi_func = sp.lambdify((xi, eta), dN_bar_dxi, 'numpy')
    dN_bar_deta_func = sp.lambdify((xi, eta), dN_bar_deta, 'numpy')

    dN_bar_dxi = dN_bar_dxi_func(xi_vals, eta_vals) 
    dN_bar_deta = dN_bar_deta_func(xi_vals, eta_vals)

    dN_bar_dxieta = np.vstack([dN_bar_dxi, dN_bar_deta])

    J = J_func(xi_vals, eta_vals)
    detJ = np.linalg.det(J)

    dN_bar_dxy = np.linalg.inv(J.T) @ dN_bar_dxieta
    
    Be = np.zeros((3, 8))
    for i in range(4):
        dN_dx = dN_bar_dxy[0, i]
        dN_dy = dN_bar_dxy[1, i]
        
        id_dx = 2 * i 
        id_dy = 2 * i + 1
        
        Be[0, id_dx] = dN_dx
        Be[1, id_dy] = dN_dy
        Be[2, id_dx] = dN_dx
        Be[2, id_dy] = dN_dy
        
    return N_bar, Be, detJ

x_e = sp.Matrix([
    [0.014],
    [0.021],
    [0.018],
    [0.012]
])

y_e = sp.Matrix([
    [0.010],
    [0.009],
    [0.018],
    [0.016]
])

nodes = np.hstack([x_e, y_e])

Ne, Be, detJ = compute_Ne_Be_detJ(nodes, 1, 1)

displayvar('Be',Be)


a)
4.999999999999997e-05
b)


<IPython.core.display.Math object>

In [ ]:
# Problem 5

# Load the mesh from file
import pickle

filename = "heat_flow.pkl"

with open(filename, "rb") as f:
    mesh = pickle.load(f)

fig = plot_mesh(mesh.nodes, mesh.elements, mesh.edges, 
show_node_ids=True)
fig.show()

displayvar("\\text{edge nodes in mesh}",mesh.edges)
# Input data
t = 1 # thickness [m]
k = 1 # conductivity [W/m]
T0 = 0 # Prescribed temperature on edges
h = 2 # prescribed heat flux on the inner boundary [W/m^2]

<IPython.core.display.Math object>

In [36]:
D = k * np.array([[1, 0],
                  [0, 1]])

num_nodes = mesh.nodes.shape[0]
num_el = mesh.elements.shape[0]
num_dofs = num_nodes

K = np.zeros((num_dofs, num_dofs))
f = np.zeros((num_dofs))

for el in range(num_el):  
    el_nodes = mesh.nodes[mesh.elements[el] - 1]
    dofs = mesh.edofs[el, :]
    Ke, fe = flow2t_Ke_fe(el_nodes, t, D)
    assem(K, Ke, dofs)
    assem(f, fe, dofs)
    
def fe_edge(nodes, qn, t):
    x1, y1 = nodes[0]
    x2, y2 = nodes[1]
    L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    fe_ed = t * qn * (L / 2) * np.array([[1],
                                         [1]])
    return fe_ed

inner_edges = mesh.edges['inner']
num_edges = inner_edges.shape[0] - 1

for edge in range(num_edges):
    edge_nodes = inner_edges[edge:edge + 2] - 1
    # print(f'edge nodes: {edge_nodes}')
    nodes = mesh.nodes[edge_nodes, :]
    # print(f'nodes: {nodes}')
    dofs =  edge_nodes + 1
    
    fe = fe_edge(nodes, h, t)
    assem(f, fe, dofs)

left_edges = mesh.edges['left']
right_edges = mesh.edges['right']
bc_dofs = np.hstack([left_edges, right_edges])
bc_vals = np.ones_like(bc_dofs) * T0

a, r = solve_eq(K, f, bc_dofs, bc_vals)

Edofs = mesh.edofs
Ed = extract_dofs(a, Edofs)
fig = plot_scalar_field(mesh.nodes, mesh.elements, Ed)
fig.show()